In [1]:
from libs.SatelliteEnvironment import *
from libs.DRLAgents.DQN import DQN
from torch.utils.tensorboard import SummaryWriter

import numpy as np
import torch 

In [2]:
env = CircularOrbit()

state = env.get_state()

num_states = state.shape[0]
orbit_num  = env.orbit_num
sat_num    = env.satellite_num
num_action = orbit_num * sat_num

model      = DQN(num_states = num_states,
                 num_action = num_action)

writer     = SummaryWriter('runs/satellite')
episodes   = 10000

for i in range(episodes):
    env.reset()
    ep_reward = 0
    ep_loss = 0 
    while True:
        state = env.get_state()
        
        action = model.choose_action(state)

        reward, done, info = env.step(action) # 
        
        next_state = env.get_state() # 
        
        model.store_transition(state, action, reward, next_state) # 
        ep_reward += reward

        if model.memory_counter >= MEMORY_CAPACITY:

            loss = model.learn(network_iteration= 100,
                                batch_size= 32,
                                gamma=0.9)
            ep_loss += loss
            if done:
                print("episode: {} , the episode reward is {}".format(i, round(ep_reward, 3)))
        if done:
            break

    writer.add_scalar('Episode Reward', ep_reward, i)
    writer.add_scalar('Episode Loss', ep_loss, i)

writer.close()

episode: 1012 , the episode reward is 735.729
episode: 1013 , the episode reward is 1922.284
episode: 1014 , the episode reward is 1290.205
episode: 1015 , the episode reward is 4419.67
episode: 1016 , the episode reward is 697.209
episode: 1017 , the episode reward is 945.47
episode: 1018 , the episode reward is -100
episode: 1019 , the episode reward is -100
episode: 1020 , the episode reward is -100
episode: 1021 , the episode reward is -100
episode: 1022 , the episode reward is -100
episode: 1023 , the episode reward is -100
episode: 1024 , the episode reward is 5650.905
episode: 1025 , the episode reward is 341.471
episode: 1026 , the episode reward is 1418.223
episode: 1027 , the episode reward is -100
episode: 1028 , the episode reward is -100
episode: 1029 , the episode reward is -100
episode: 1030 , the episode reward is -100
episode: 1031 , the episode reward is 655.182
episode: 1032 , the episode reward is 429.809
episode: 1033 , the episode reward is -100
episode: 1034 , th

ValueError: math domain error